<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/FINAL/Code/dropout/sparsity_dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import pickle
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
torch.manual_seed(1234)
np.random.seed(1234)

In [4]:
mnist_trainset = datasets.MNIST(root='./data', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = datasets.MNIST(root='./data', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=50, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

print("Training dataset size: ", len(mnist_trainset))
print("Testing dataset size: ",  len(mnist_testset))

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!
Training dataset size:  60000
Testing dataset size:  10000


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
# Define the model 
class Model(torch.nn.Module):
    def __init__(self, dropout_rate):
        super(Model, self).__init__()
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 10)
        self.sigmoid  = torch.nn.Sigmoid()
        self.dropout  = torch.nn.Dropout(p=dropout_rate)

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid(x)
        x = self.dropout(x)
        pred = self.linear_2(x)

        return pred

In [6]:
def get_activation(model):    
    def hook(module, input, output):
        model.layer_activations = output
    return hook

In [7]:
no_epochs = 30
criterion = torch.nn.CrossEntropyLoss()
def sparsity_trainer(optimizer, model):

    # reset the model 
    train_loss = list()
    test_loss  = list()
    test_acc   = list()
    final_spareness = list()

    # define activation list 

    best_test_loss = 1

    for epoch in range(no_epochs):
        total_train_loss = 0
        total_test_loss = 0

        hidden_layer_activation_list = list()

        # training
        # set up training mode 
        model.train()
        for itr, (image, label) in enumerate(train_dataloader):

            image, label = image.to(device), label.to(device)

            optimizer.zero_grad()

            pred = model(image)

            loss = criterion(pred, label)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (image, label) in enumerate(test_dataloader):
            image, label = image.to(device), label.to(device)

            pred = model(image)

            loss = criterion(pred, label)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if label[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            hidden_layer_activation_list.append(model.layer_activations)

        # this conains activations for all epochs 
        final_spareness.append(hidden_layer_activation_list)

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

        # if total_test_loss < best_test_loss:
        #     best_test_loss = total_test_loss
            # print("Saving the model state dictionary for Epoch: {} with Test loss: {:.8f}".format(epoch + 1, total_test_loss))
            # torch.save(model.state_dict(), "model.dth")

    sparseness_list = list()

    for single_epoch_spareness in final_spareness:
        single_epoch_spareness = torch.stack(single_epoch_spareness)
        layer_activations_list = torch.reshape(single_epoch_spareness, (10000, 256))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)
        mean_sparseness_per_epoch = torch.mean(population_sparseness)

        sparseness_list.append(mean_sparseness_per_epoch)

    return test_acc, sparseness_list

In [8]:
dropout_list = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# AdaDelta

In [ ]:
for i in range(10):
    print(f"Entering {i}-th loop")
    model_Adadelta = Model(dropout_list[i])
    print("dropout rate:", model_Adadelta.dropout)
    model_Adadelta.to(device)
    model_Adadelta.sigmoid.register_forward_hook(get_activation(model_Adadelta))
    optimizer_Adadelta = torch.optim.Adadelta(model_Adadelta.parameters(), lr=1.0)
    Adadelta_test_acc, Adadelta_sparseness_list = sparsity_trainer(optimizer=optimizer_Adadelta, model=model_Adadelta)
    
    if i == 0:
        f = open("dropout_sparsity_Adadelta.txt", "w")
        f.write(str(i)+'\n'+str(Adadelta_test_acc)+'\n'+str(Adadelta_sparseness_list)+'\n\n')
    else:
        f = open("dropout_sparsity_Adadelta.txt", "a")
        f.write(str(i)+'\n'+str(Adadelta_test_acc)+'\n'+str(Adadelta_sparseness_list)+'\n\n')
f.close()

!cp dropout_sparsity_Adadelta.txt /content/drive/MyDrive

Entering 0-th loop
dropout rate: Dropout(p=0.0, inplace=False)

Epoch: 1/30, Train Loss: 0.43482732, Test Loss: 0.25056327, Test Accuracy: 0.92680000

Epoch: 2/30, Train Loss: 0.21552303, Test Loss: 0.17478676, Test Accuracy: 0.94670000

Epoch: 3/30, Train Loss: 0.15641611, Test Loss: 0.13820601, Test Accuracy: 0.95740000

Epoch: 4/30, Train Loss: 0.12157688, Test Loss: 0.11924305, Test Accuracy: 0.96300000

Epoch: 5/30, Train Loss: 0.09919054, Test Loss: 0.09831682, Test Accuracy: 0.96860000

Epoch: 6/30, Train Loss: 0.08336798, Test Loss: 0.09016021, Test Accuracy: 0.97190000

Epoch: 7/30, Train Loss: 0.07088852, Test Loss: 0.08719795, Test Accuracy: 0.97320000

Epoch: 8/30, Train Loss: 0.06153277, Test Loss: 0.07796416, Test Accuracy: 0.97520000

Epoch: 9/30, Train Loss: 0.05413193, Test Loss: 0.07215949, Test Accuracy: 0.97700000

Epoch: 10/30, Train Loss: 0.04688472, Test Loss: 0.07165622, Test Accuracy: 0.97650000

Epoch: 11/30, Train Loss: 0.04203055, Test Loss: 0.07021960, Test

# AdaGrad

In [ ]:
for i in range(10):
    print(f"Entering {i}-th loop")
    model_Adagrad = Model(dropout_list[i])
    print("dropout rate:", model_Adagrad.dropout)
    model_Adagrad.to(device)
    model_Adagrad.sigmoid.register_forward_hook(get_activation(model_Adagrad))
    optimizer_Adagrad = torch.optim.Adagrad(model_Adagrad.parameters(), lr=0.1)
    Adagrad_test_acc, Adagrad_sparseness_list = sparsity_trainer(optimizer=optimizer_Adagrad, model=model_Adagrad)
    
    if i == 0:
        f = open("dropout_sparsity_Adagrad.txt", "w")
        f.write(str(i)+'\n'+str(Adagrad_test_acc)+'\n'+str(Adagrad_sparseness_list)+'\n\n')
    else:
        f = open("dropout_sparsity_Adagrad.txt", "a")
        f.write(str(i)+'\n'+str(Adagrad_test_acc)+'\n'+str(Adagrad_sparseness_list)+'\n\n')
f.close()

!cp dropout_sparsity_Adagrad.txt /content/drive/MyDrive

# SGD

In [ ]:
for i in range(10):
    print(f"Entering {i}-th loop")
    model_SGD = Model(dropout_list[i])
    print("dropout rate:", model_SGD.dropout)
    model_SGD.to(device)
    model_SGD.sigmoid.register_forward_hook(get_activation(model_SGD))
    optimizer_SGD = torch.optim.SGD(model_SGD.parameters(), lr=0.1)
    SGD_test_acc, SGD_sparseness_list = sparsity_trainer(optimizer=optimizer_SGD, model=model_SGD)
    
    if i == 0:
        f = open("dropout_sparsity_SGD.txt", "w")
        f.write(str(i)+'\n'+str(SGD_test_acc)+'\n'+str(SGD_sparseness_list)+'\n\n')
    else:
        f = open("dropout_sparsity_SGD.txt", "a")
        f.write(str(i)+'\n'+str(SGD_test_acc)+'\n'+str(SGD_sparseness_list)+'\n\n')
f.close()

!cp dropout_sparsity_SGD.txt /content/drive/MyDrive

# Adam

In [ ]:
for i in range(10):
    print(f"Entering {i}-th loop")
    model_Adam = Model(dropout_list[i])
    print("dropout rate:", model_Adam.dropout)
    model_Adam.to(device)
    model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
    optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
    Adam_test_acc, Adam_sparseness_list = sparsity_trainer(optimizer=optimizer_Adam, model=model_Adam)
    
    if i == 0:
        f = open("dropout_sparsity_Adam.txt", "w")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(Adam_sparseness_list)+'\n\n')
    else:
        f = open("dropout_sparsity_Adam.txt", "a")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(Adam_sparseness_list)+'\n\n')
f.close()

!cp dropout_sparsity_Adam.txt /content/drive/MyDrive